## Enhanced featrue engineer model

This model will add engineered features for the original question, in addition to the lemmatized question.

Engineered two different types of features,

1. n_gram similarity between each pair of questions
2. min/max/avg distance between words in a single question. Currently using the following metrics,
  * euclidean
  * cosine
  * city block or manhattan
  
**Pipeline**
1. Stack questions
2. Clean questions - now lower cases all words to better lemmatize proper nouns
3. UNION
    1. n_gram similarity
    2. min/max/avg distance
4. Lemmatize questions
5. UNION
    1. n_gram similarity
    2. min/max/avg distances
6. UNION together both sets of features
7. Random Forrest

In [1]:
# data manipulation
import utils
import pandas as pd
import numpy as np

# modeling
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [2]:
X_train = utils.load('X_train')
y_train = utils.load('y_train')

In [3]:
# feature engineering pipes
single_question_pipe = Pipeline(
    [
        ('dist', FunctionTransformer(utils.add_min_max_avg_distance_features, validate=False)),
        ('unstack', FunctionTransformer(utils.unstack_questions, validate=False))
    ]
)

pair_question_pipe = Pipeline(
    [
        ('ngram_sim', FunctionTransformer(utils.calc_ngram_similarity, kw_args={'n_grams':[1, 2, 3]}, validate=False))
    ]
)

# clean text pipe
clean_text_pipe = Pipeline(
    [
        ('stack', FunctionTransformer(utils.stack_questions, validate=False)),
        ('clean', FunctionTransformer(utils.clean_questions, validate=False)),
        ('feats', FeatureUnion(
            [
                ('pair', pair_question_pipe),
                ('single', single_question_pipe)
            ]
        ))
    ]
)

# lemma pipe
lemma_pipe = Pipeline(
    [
        ('stack', FunctionTransformer(utils.stack_questions, validate=False)),
        ('clean', FunctionTransformer(utils.clean_questions, validate=False)),
        ('lemma', FunctionTransformer(utils.apply_lemma, validate=False)),
        ('feats', FeatureUnion(
            [
                ('pair', pair_question_pipe),
                ('single', single_question_pipe)
            ]
        ))
    ]
)

# pre-process pipe
pre_process_pipe = Pipeline(
    [
        ('feats', FeatureUnion(
            [
                ('clean_features', clean_text_pipe),
                ('lemma_pipe', lemma_pipe)
            ]
        ))
    ]
)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)

In [4]:
X_transform = pre_process_pipe.transform(X_train)

skf = StratifiedKFold(n_splits=3, random_state=42)
cv = cross_validate(rf, 
               X_transform, 
               y_train, 
               cv=skf, 
               n_jobs=-1, 
               scoring=('accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'neg_log_loss'))

In [5]:
results_df = utils.load('results')

results_df = results_df.drop(index='rf_feat_eng_model_lemma_clean', errors='ignore')
results_df = results_df.append(utils.log_scores(cv, 'rf_feat_eng_model_lemma_clean'))
results_df

,avg_accuracy,std_accuracy,avg_precision,std_precision,avg_recall,std_recall,avg_f1,std_f1,avg_auc,std_auc,avg_log_loss,std_log_loss
"mvp (tf-idf, nmf(5), xgboost)",0.700345,0.000466,0.661571,0.000461,0.385736,0.002493,0.487325,0.001983,0.740593,0.001647,0.568958,0.001288
mvp (+ lemma),0.696787,0.001055,0.649977,0.003057,0.387424,0.003230,0.485464,0.002485,0.738037,0.001362,0.572483,0.000815
cos_sim_model,0.710200,0.000830,0.658748,0.002578,0.446336,0.002215,0.532120,0.001306,0.746769,0.001279,0.565250,0.000963
cos_sim_tfidf_model,0.728261,0.001248,0.659662,0.002240,0.545419,0.001370,0.597124,0.001666,0.799173,0.001407,0.513172,0.001191
feat_eng_model,0.743614,0.002021,0.664102,0.003502,0.618400,0.001553,0.640434,0.002281,0.821070,0.001428,0.489465,0.001141
feat_eng_model_lemma_fix,0.744356,0.002107,0.664513,0.004333,0.621357,0.000901,0.642201,0.001609,0.822197,0.001710,0.488131,0.001342
feat_eng_model_lemma_clean,0.763927,0.002404,0.676166,0.003904,0.692113,0.001128,0.684044,0.002549,0.846923,0.001643,0.456929,0.001410
rf_feat_eng_model_lemma_clean,0.783667,0.002260,0.708853,0.003681,0.702725,0.001666,0.705774,0.002658,0.868202,0.001148,0.436197,0.000640


In [6]:
utils.save(results_df, 'results')

## Results

Wow! The feature engineering shows a significant jump in AUC from 0.8 to 0.82.

In [7]:
rf.fit(X_transform, y_train)
y_probs = rf.predict_proba(X_transform)[:, 1]
class_errors_df = utils.ground_truth_analysis(y_train, y_probs)
class_errors_df.head()

,gt,prob,diff
0,0,0.150,-0.150
1,0,0.180,-0.180
2,0,0.086,-0.086
3,0,0.000,0.000
4,0,0.042,-0.042


In [8]:
lemma_pipe = Pipeline(
    [
        ('stack', FunctionTransformer(utils.stack_questions, validate=False)),
        ('clean', FunctionTransformer(utils.clean_questions, validate=False)),
        ('lemma', FunctionTransformer(utils.apply_lemma, validate=False)),
    ]
)
X_train_lemma = lemma_pipe.transform(X_train)

## Top false negative errors

In [9]:
fn_idx = class_errors_df.sort_values('diff', ascending = False).head().index
for idx in fn_idx:
    print('Prob:', y_probs[idx])
    print()
    print(X_train.iloc[idx].question1)
    print(X_train.iloc[idx].question2)
    print()
    print('Lemma--------')
    print()
    print(X_train_lemma[idx*2])
    print(X_train_lemma[idx*2+1])
    print()
    print('Feature Space------')
    print(X_transform[idx])
    print('-------------------------------------------')
    print()

Prob: 0.022035150817837166

Is unifunds.com legit?
Is unifunds legit?

Lemma--------

unifund legit
unifundscom legit

Feature Space------
[ 0.66666667  0.33333333  0.33333333  6.57793499  6.57793499  6.57793499
  0.732846    0.732846    0.732846   88.33192412 88.33192412 88.33192412
  6.57793499  6.57793499  6.57793499  0.732846    0.732846    0.732846
 88.33192412 88.33192412 88.33192412  0.5         0.5         0.5
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
-------------------------------------------

Prob: 0.06587658468825115

What is the remainder when 2^100 is divided by 101?
What is the remainder when [math]2^{100}[/math] is divided by 101?

Lemma--------

remainder  divide
remainder  divide

Feature Space------
[  1.           1.           1.           0.           7.61300637
   5.48716164   0.           0.83449638   0.

## Top false positive errors

In [10]:
fn_idx = class_errors_df.sort_values('diff').head().index
for idx in fn_idx:
    print('Prob:', y_probs[idx])
    print()
    print(X_train.iloc[idx].question1)
    print(X_train.iloc[idx].question2)
    print()
    print('Lemma--------')
    print()
    print(X_train_lemma[idx*2])
    print(X_train_lemma[idx*2+1])
    print()
    print('Feature Space------')
    print(X_transform[idx])
    print('-------------------------------------------')
    print()

Prob: 0.859919422244422

Can I make 30,000 a month betting on horses?
Can I make 80,000 a month betting on horses?

Lemma--------

 month bet horse
 month bet horse

Feature Space------
[  1.           1.           1.           3.54843587   8.64389945
   6.99204975   0.24125347   0.86209143   0.6860896   48.7158108
 119.75634825  95.64765528   3.54843587   8.64389945   6.99204975
   0.24125347   0.86209143   0.6860896   48.7158108  119.75634825
  95.64765528   1.           1.           1.           6.92833324
   8.06033018   7.32694928   0.60332816   0.81230419   0.70401332
  95.01284819 112.79109566 101.19969094   6.92833324   8.06033018
   7.32694928   0.60332816   0.81230419   0.70401332  95.01284819
 112.79109566 101.19969094]
-------------------------------------------

Prob: 0.8402367798867798

What was the significance of the battle of Somme, and how did this battle compare and contrast to the Battle of Taiyuan?
What was the significance of the battle of Somme, and how did this 

## Next Steps

1. The set of false negative and false postive errors appears to be different compared to XGBoost model. 
  * Run an ensemble model with RandomForrest and XGBoost to smooth out the bias.
2. Short questions with numbers and stop words seem to be a common theme for the false positive errors. 
  * Generate features including stop words and numbers.